In [97]:
# Import all the helper functions
from helper import *

## 2. Explorative Datenanalyse

### 2.1 CO2 Emissionen

#### 2.1.1 Daten Bereinigung

Hier lesen wir die CO2 Emissionen ein und bearbeiten die Daten wie folgt:

1. Behalte Einträge ab Jahr 1990 im DataFrame.
2. Entferne die Spalte "Code".
3. Ändere den Spaltennamen "Entity" in "Country".
4. Behalte nur Schweiz, Spanien, Deutschland und UK.
5. Teile Werte in der Spalte "Annual CO₂ emissions" in miliarde Tonnen um.

In [98]:
# Read co2 emission data
df_emissions = pd.read_csv("Data/All/CO2/annual-co2-emissions-per-country.csv")

# Wrangle data
df_emissions = wrangle_dataframe(df_emissions, "Annual CO₂ emissions")

# Rename Annual CO₂ emissions to CO2 emissions (billion t)
df_emissions.rename(columns = {'Annual CO₂ emissions': 'CO₂ emissions (billion t)'}, inplace = True)

df_emissions

,Country,Year,CO₂ emissions (billion t)
0,Germany,1990,1.051979
1,Germany,1991,1.013824
2,Germany,1992,0.965542
3,Germany,1993,0.955820
4,Germany,1994,0.939492
...,...,...,...
123,United Kingdom,2017,0.387367
124,United Kingdom,2018,0.379730
125,United Kingdom,2019,0.364753
126,United Kingdom,2020,0.326263


### 2.1.2 CO2 Emissionen pro Kopf

Es ist aber nicht sinnvoll, die absoluten CO2 Emissionen zu betrachten, da die Länder unterschiedlich gross in Landesfläche und Population sind. Deshalb betrachten wir die CO2-Emissionen pro Kopf und Jahr sowie pro Quadratmeter und Jahr. Dies ermöglicht einen Vergleich der Emissionen, der unabhängig von der Grösse des Landes und der Bevölkerung ist.

Um die CO2-Emissionen pro Kopf und Jahr zu berechnen, teilen wir die Gesamtemissionen eines Landes in einem bestimmten Jahr durch die Bevölkerungszahl dieses Landes in diesem Jahr.

#### 2.1.2.1 Datenaufbereitung

In [99]:
# Read population data
df_population = pd.read_csv("Data/All/population.csv")

# Rename population column
df_population.rename(columns = {'Population (historical estimates)': 'Population'}, inplace = True)

# Wrangle data
df_population = wrangle_dataframe(df_population, "Population")

df_population

,Country,Year,Population
0,Germany,1990,79370192
1,Germany,1991,79801984
2,Germany,1992,80205024
3,Germany,1993,80573496
4,Germany,1994,80882120
...,...,...,...
123,United Kingdom,2017,66064808
124,United Kingdom,2018,66432996
125,United Kingdom,2019,66778660
126,United Kingdom,2020,67059472


Wir können nun die beiden Datensätzen verbinden und die Emissionen pro Kopf und Jahr berechnen.

In [100]:
# Merge dataframes
df_emissions_population = pd.merge(df_emissions, df_population, on = ['Country', 'Year'])

# Create copy of dataframe for emissions per capita
df_em_pop_cap = df_emissions_population.copy()

# Calculate emissions per capita
df_em_pop_cap['CO₂ emissions per capita (t)'] = round(df_em_pop_cap['CO₂ emissions (billion t)'] / df_em_pop_cap['Population'] * 1000000000, 2)

df_em_pop_cap

,Country,Year,CO₂ emissions (billion t),Population,CO₂ emissions per capita (t)
0,Germany,1990,1.051979,79370192,13.25
1,Germany,1991,1.013824,79801984,12.70
2,Germany,1992,0.965542,80205024,12.04
3,Germany,1993,0.955820,80573496,11.86
4,Germany,1994,0.939492,80882120,11.62
...,...,...,...,...,...
123,United Kingdom,2017,0.387367,66064808,5.86
124,United Kingdom,2018,0.379730,66432996,5.72
125,United Kingdom,2019,0.364753,66778660,5.46
126,United Kingdom,2020,0.326263,67059472,4.87


Jetzt haben wir die CO₂ Emissionen pro Kopf pro Land und Jahr. Wir können nun die Entwicklung der Emissionen pro Kopf pro Land über die Zeit betrachten (absolute und relative Werte).

In [101]:
# Create new column and group by country to get value from first row (1990) - add that value to each row
df_em_pop_cap['CO₂ emissions 1990'] = df_em_pop_cap.groupby('Country')['CO₂ emissions (billion t)'].transform(lambda x: x.iloc[0])

# Calculate relative change since 1990
df_em_pop_cap['Relative change CO₂ since 1990'] = round((df_em_pop_cap['CO₂ emissions (billion t)'] - df_em_pop_cap['CO₂ emissions 1990']) / df_em_pop_cap['CO₂ emissions 1990'] * 100, 2)

df_em_pop_cap

,Country,Year,CO₂ emissions (billion t),Population,CO₂ emissions per capita (t),CO₂ emissions 1990,Relative change CO₂ since 1990
0,Germany,1990,1.051979,79370192,13.25,1.051979,0.00
1,Germany,1991,1.013824,79801984,12.70,1.051979,-3.63
2,Germany,1992,0.965542,80205024,12.04,1.051979,-8.22
3,Germany,1993,0.955820,80573496,11.86,1.051979,-9.14
4,Germany,1994,0.939492,80882120,11.62,1.051979,-10.69
...,...,...,...,...,...,...,...
123,United Kingdom,2017,0.387367,66064808,5.86,0.601945,-35.65
124,United Kingdom,2018,0.379730,66432996,5.72,0.601945,-36.92
125,United Kingdom,2019,0.364753,66778660,5.46,0.601945,-39.40
126,United Kingdom,2020,0.326263,67059472,4.87,0.601945,-45.80


In [102]:
# Create new column and group by country to get value from first row (1990) - add that value to each row
df_em_pop_cap['CO₂ emissions per capita 1990'] = df_em_pop_cap.groupby('Country')['CO₂ emissions per capita (t)'].transform(lambda x: x.iloc[0])

# Calculate relative change since 1990
df_em_pop_cap['Relative change CO₂ per capita since 1990'] = round((df_em_pop_cap['CO₂ emissions per capita (t)'] - df_em_pop_cap['CO₂ emissions per capita 1990']) / df_em_pop_cap['CO₂ emissions per capita 1990'] * 100, 2)

# rename country to Land
df_em_pop_cap.rename(columns = {'Country': 'Land'}, inplace = True)

# rename Year to Jahr
df_em_pop_cap.rename(columns = {'Year': 'Jahr'}, inplace = True)

df_em_pop_cap

,Land,Jahr,CO₂ emissions (billion t),Population,CO₂ emissions per capita (t),CO₂ emissions 1990,Relative change CO₂ since 1990,CO₂ emissions per capita 1990,Relative change CO₂ per capita since 1990
0,Germany,1990,1.051979,79370192,13.25,1.051979,0.00,13.25,0.00
1,Germany,1991,1.013824,79801984,12.70,1.051979,-3.63,13.25,-4.15
2,Germany,1992,0.965542,80205024,12.04,1.051979,-8.22,13.25,-9.13
3,Germany,1993,0.955820,80573496,11.86,1.051979,-9.14,13.25,-10.49
4,Germany,1994,0.939492,80882120,11.62,1.051979,-10.69,13.25,-12.30
...,...,...,...,...,...,...,...,...,...
123,United Kingdom,2017,0.387367,66064808,5.86,0.601945,-35.65,10.52,-44.30
124,United Kingdom,2018,0.379730,66432996,5.72,0.601945,-36.92,10.52,-45.63
125,United Kingdom,2019,0.364753,66778660,5.46,0.601945,-39.40,10.52,-48.10
126,United Kingdom,2020,0.326263,67059472,4.87,0.601945,-45.80,10.52,-53.71


### 2.1.3 CO2 Emissionen pro Quadratmeter

Hier die Landesflächen der Länder:

| Land | Fläche in km2 |
| --- | --- |
| Deutschland | 357.386 |
| Spanien | 505.990 |
| Schweiz | 41.285 |
| UK | 242.495 |

[Quelle](https://de.wikipedia.org/wiki/Liste_von_Staaten_und_Territorien_nach_Fl%C3%A4che)

Um die CO2-Emissionen pro Quadratmeter und Jahr zu berechnen, teilen wir die Gesamtemissionen eines Landes in einem bestimmten Jahr durch die Fläche dieses Landes.

#### 2.1.3.1 Datenaufbereitung

In [103]:
land_areas = [357386, 505990, 41285, 242495]

# Get list of countries
countries = df_em_pop_cap['Land'].unique().tolist()

# Match countries with land areas
country_land_area = dict(zip(countries, land_areas))

# Create new dataframe for emissions per land area
df_em_land = df_em_pop_cap.copy()

df_em_land['Land Area'] = df_em_land['Land'].map(country_land_area)
df_em_land['CO₂ emissions per land area (t/km²)'] = round(df_em_land['CO₂ emissions (billion t)'] * 1e9 / df_em_land['Land Area'], 2)

# Remove column Land Area
df_em_land.drop(columns = [
    'Land Area', 
    'Population', 
    'CO₂ emissions per capita (t)', 
    'CO₂ emissions 1990',
    'Relative change CO₂ since 1990',
    'CO₂ emissions per capita 1990',
    'Relative change CO₂ per capita since 1990'
    ]
    , inplace = True
)

# rename column CO₂ emissions (billion t) to CO₂ Emissionen (milliard t/km²)
df_em_land.rename(columns = {'CO₂ emissions (billion t)': 'CO₂ emissions (milliard t/km²)'}, inplace = True)

# rename column CO₂ emissions per land area (t/km²) to CO₂ Emissionen pro Landfläche (t/km²)
df_em_land.rename(columns = {'CO₂ emissions per land area (t/km²)': 'CO₂ emissions pro Landfläche (t/km²)'}, inplace = True)

df_em_land

,Land,Jahr,CO₂ emissions (milliard t/km²),CO₂ emissions pro Landfläche (t/km²)
0,Germany,1990,1.051979,2943.54
1,Germany,1991,1.013824,2836.78
2,Germany,1992,0.965542,2701.68
3,Germany,1993,0.955820,2674.48
4,Germany,1994,0.939492,2628.79
...,...,...,...,...
123,United Kingdom,2017,0.387367,1597.42
124,United Kingdom,2018,0.379730,1565.93
125,United Kingdom,2019,0.364753,1504.17
126,United Kingdom,2020,0.326263,1345.44


## 2.2 Erneuerbare Energien
### 2.2.1 Datenaufbereitung

Diese Daten wurden auf die gleiche Weise wie die CO2-Emissionsdaten aufbereitet. "add_change_columns" berechnet den jährlichen absoluten und relativen Anstieg oder Abstieg der Energieerzeugung aus erneuerbaren Quellen.

In [104]:
df_renewable_energy = pd.read_csv('Data/All/modern-renewable-prod.csv')

df_renewable_energy = wrangle_dataframe(df_renewable_energy, None)

# rename Country to Land
df_renewable_energy.rename(columns = {'Country': 'Land'}, inplace = True)

# rename column Year to Jahr
df_renewable_energy.rename(columns = {'Year': 'Jahr'}, inplace = True)

# rename Electricity from wind to Produktion Windenergie (TWh)
df_renewable_energy.rename(columns = {'Electricity from wind (TWh)': 'Produktion Windenergie (TWh)'}, inplace = True)

# rename Electricity from solar to Produktion Solarenergie (TWh)
df_renewable_energy.rename(columns = {'Electricity from solar (TWh)': 'Produktion Solarenergie (TWh)'}, inplace = True)

# rename Electricity from hydro to Produktion Wasserkraft (TWh)
df_renewable_energy.rename(columns = {'Electricity from hydro (TWh)': 'Produktion Wasserkraft (TWh)'}, inplace = True)

# rename Electricity from geothermal to Produktion Geothermie (TWh)
df_renewable_energy.rename(columns = {'Other renewables including bioenergy (TWh)': 'Produktion Biomasse (TWh)'}, inplace = True)

df_renewable_energy

,Land,Jahr,Produktion Windenergie (TWh),Produktion Wasserkraft (TWh),Produktion Solarenergie (TWh),Produktion Biomasse (TWh)
0,Germany,1990,0.07,17.43,0.00,1.60
1,Germany,1991,0.22,14.89,0.00,1.87
2,Germany,1992,0.29,17.40,0.00,2.04
3,Germany,1993,0.67,17.88,0.00,1.97
4,Germany,1994,1.43,19.93,0.01,2.07
...,...,...,...,...,...,...
125,United Kingdom,2017,49.64,5.88,11.46,31.87
126,United Kingdom,2018,56.91,5.44,12.74,34.94
127,United Kingdom,2019,64.33,5.93,12.92,37.30
128,United Kingdom,2020,73.64,6.68,13.32,38.10


# Widgets

In [105]:
# Create interactive dataframes
interactive_co2_capita = df_em_pop_cap.interactive()
interactive_co2_land_area = df_em_land.interactive()
interactive_renewable_energy = df_renewable_energy.interactive()
interactive_renewable_energy_new = df_renewable_energy.interactive()


# Get list of countries
countries = list(df_renewable_energy["Land"].unique())

# Create country selector widget
country_selector = pn.widgets.CheckButtonGroup(
    name = 'Country selector', 
    value = countries, 
    options = countries,
    button_type = 'primary',
    sizing_mode = 'stretch_both'
)

renewable_energy_selector = pn.widgets.RadioButtonGroup(
    name = 'Renewable energy selector', 
    options = ["Produktion Windenergie (TWh)", "Produktion Wasserkraft (TWh)", "Produktion Solarenergie (TWh)"],
    button_type = 'primary'
)

# Create year slider widget "Fron"
start_year_slider = pn.widgets.IntSlider(
    name = 'Von ',
    start = 1990,
    end = 2020,
    step = 1,
    value = 1990
)

# Create year slider widget "Until"
end_year_slider = pn.widgets.IntSlider(
    name = 'Bis ',
    start = 1990,
    end = 2020,
    step = 1,
    value = 2020
)

# Pipes

In [106]:
df_renewable_energy

,Land,Jahr,Produktion Windenergie (TWh),Produktion Wasserkraft (TWh),Produktion Solarenergie (TWh),Produktion Biomasse (TWh)
0,Germany,1990,0.07,17.43,0.00,1.60
1,Germany,1991,0.22,14.89,0.00,1.87
2,Germany,1992,0.29,17.40,0.00,2.04
3,Germany,1993,0.67,17.88,0.00,1.97
4,Germany,1994,1.43,19.93,0.01,2.07
...,...,...,...,...,...,...
125,United Kingdom,2017,49.64,5.88,11.46,31.87
126,United Kingdom,2018,56.91,5.44,12.74,34.94
127,United Kingdom,2019,64.33,5.93,12.92,37.30
128,United Kingdom,2020,73.64,6.68,13.32,38.10


In [107]:
renewable_energy_pipeline = (
    interactive_renewable_energy[ 
        (interactive_renewable_energy.Land.isin(country_selector)) &
        (interactive_renewable_energy.Jahr.astype('int') >= start_year_slider) &
        (interactive_renewable_energy.Jahr.astype('int') <= end_year_slider)
    ]
    .reset_index(drop=True)

)
co2_capita_pipeline = (
    interactive_co2_capita[
        (interactive_co2_capita.Land.isin(country_selector)) &
        (interactive_co2_capita.Jahr.astype('int') >= start_year_slider) &
        (interactive_co2_capita.Jahr.astype('int') <= end_year_slider)
    ]
    .reset_index(drop=True)
)

co2_land_area_pipeline = (
    interactive_co2_land_area[
        (interactive_co2_land_area.Land.isin(country_selector)) &
        (interactive_co2_land_area.Jahr.astype('int') >= start_year_slider) &
        (interactive_co2_land_area.Jahr.astype('int') <= end_year_slider)
    ]
    .reset_index(drop=True)
)

renewable_energy_pipeline_new = (
    interactive_renewable_energy_new[ 
        (interactive_renewable_energy_new.Land.isin(country_selector)) &
        (interactive_renewable_energy_new.Jahr.astype('int') >= start_year_slider) &
        (interactive_renewable_energy_new.Jahr.astype('int') <= end_year_slider)
    ]
    .reset_index(drop=True)
)

# Plots

In [108]:
# Plot Co2 emissions per capita
plot_co2 = co2_capita_pipeline.hvplot(
    x = 'Jahr', 
    y = 'CO₂ emissions per capita (t)',
    by = 'Land', 
    title = 'Entwicklung der CO₂ Emissionen pro Kopf',
    ylabel = 'CO₂ Emissionen pro Kopf (t)',
    width = 700, height = 400)

# Plot Co2 emissions per land area
plot_co2_km2 = co2_land_area_pipeline.hvplot(
    x = 'Jahr', 
    y = 'CO₂ emissions pro Landfläche (t/km²)',
    by = 'Land', 
    title = 'Entwicklung der CO₂ Emissionen pro Landesfläche',
    ylabel = 'CO₂ Emissions pro Landesfläche (t/km²)',
    width = 700, height = 400)

# Plot Solar Energy Production
plot_solar = renewable_energy_pipeline.hvplot(       
    x = 'Jahr', 
    y = 'Produktion Solarenergie (TWh)',
    by = 'Land',
    title  = 'Solar Energieproduktion',
    xlabel = 'Jahr',
    width = 700, height = 400)

# Plot Wind Energy Production
plot_wind = renewable_energy_pipeline.hvplot(
    x = 'Jahr', 
    y = 'Produktion Windenergie (TWh)',
    by = 'Land',
    title  = 'Wind Energieproduktion',
    width = 700, height = 400)

# Plot Hydro Energy Production
plot_hydro = renewable_energy_pipeline.hvplot(
    x = 'Jahr', 
    y = 'Produktion Wasserkraft (TWh)',
    by = 'Land',
    title  = 'Wasser Energieproduktion',
    width = 700, height = 400)

plot_renewable_energy = renewable_energy_pipeline_new.hvplot(
    x = 'Jahr',
    y = renewable_energy_selector,
    by = 'Land',
    title = 'Produktion erneuerbarer Energien',
    ylabel = 'Energieproduktion (TWh)',
    width = 900, height = 400)

# Texte

In [109]:
def textblock(text, title = "", subtitle = ""):

    return (
        f"""
        \n### {title}
        \n#### {subtitle}
        \n {text}
        """
    )

In [118]:
text_allgemeine_infos = """In diesem Dashboard befinden sich Daten bezüglich der Solar-, Wind- und Wasserenergie. Nebenbei werden noch Daten der CO2 Emissionen vorgestellt. Eingeschränkt wurde das Ganze auf die Länder: Deutschland, Spanien, Schweiz und Grossbritannien"""

text_einleitung = """
Willkommen zu unserem interaktiven Dashboard, das sich der Analyse erneuerbarer Energien - insbesondere Solar-, Wasser- und Windenergie - in den Ländern Deutschland, Grossbritannien, Schweiz und Spanien widmet. In einer Ära, in der der Übergang zu erneuerbaren Energien immer wichtiger wird, ist es entscheidend, Daten und Informationen zu erfassen und zu analysieren, um fundierte Entscheidungen und Strategien zu entwickeln.

Dieses Dashboard ist ein tolles Werkzeug, mit dem Sie tiefe Einblicke in die Energiesektoren dieser vier Länder erhalten. Sie können die Daten unabhängig voneinander filtern, um spezifische Trends und Muster zu erkunden. Es ist uns ein grosses Anliegen, dass Sie die Flexibilität haben, Ihre Untersuchungen nach Belieben zu gestalten und zu kontrollieren.

Zusätzlich zu den Daten und Diagrammen, die wir zur Verfügung stellen, haben wir auch eine Datastory erstellt. Diese nimmt Sie mit auf eine Reise durch die Höhen und Tiefen, Siege und Herausforderungen der erneuerbaren Energien in diesen Ländern. Die Datastory kombiniert Datenanalyse mit Narration, um die Daten zum Leben zu erwecken und ein vollständiges Bild der Situation zu zeichnen.

Schliesslich werden wir unsere Policy Advices aufstellen basierend auf unseren Erkenntnissen. Dieser Teil des Dashboards ermöglicht es Ihnen, zukünftige Trends und Entwicklungen im Bereich der erneuerbaren Energien zu prognostizieren. Durch das Verständnis der gegenwärtigen Daten können wir auf mögliche zukünftige Szenarien hinweisen, und wir laden Sie ein, diese Policy Advices mit uns zu erforschen und zu diskutieren.

Wir hoffen, dass dieses Dashboard für Sie sowohl informativ als auch inspirierend ist. Wir freuen uns auf Ihre Erkenntnisse und Diskussionen, die uns dabei helfen, die Zukunft der erneuerbaren Energien in Europa besser zu verstehen und zu gestalten.
"""

text_sidebar = """
Dieser Leitfaden hilft Ihnen dabei, das volle Potenzial unseres interaktiven Dashboards zu nutzen. Hier erfahren Sie, wie Sie die verschiedenen Funktionen und Widgets verwenden können, um Ihre Datenanalyse so effizient und informativ wie möglich zu gestalten.

1. Filtern: Im Dashboards finden Sie eine Reihe von Filtern. Diese ermöglichen Ihnen, die dargestellten Daten nach spezifischen Kriterien einzugrenzen. Wählen Sie beispielsweise ein spezifisches Land aus, um nur Daten für dieses Land anzuzeigen.

2. Jahresanpassung: Unter den Filtern befindet sich ein Schieberegler zur Auswahl des Zeitraums. Bewegen Sie diesen, um Daten für einen bestimmten Zeitraum zu betrachten. Sie können den Schieberegler verschieben, um den Start- und Endpunkt Ihrer Datenauswahl anzupassen.

3. Plots analysieren: Jeder Plot auf dem Dashboard bietet spezifische Informationen zu den ausgewählten Kriterien. Bewegen Sie den Cursor über bestimmte Punkte oder Bereiche im Plot, um detaillierte Informationen anzuzeigen. Diese Funktion ist besonders nützlich, um tiefer in die Daten einzutauchen und spezifische Trends und Muster zu erkennen.
"""

# Textblock Quellen
text_quellen = """
- Our World in Data - [Annual CO₂ Emissions](https://ourworldindata.org/co2-emissions)
- Our World in Data - [Renewable Energy](https://ourworldindata.org/renewable-energy)
- Bundesnetzagentur - [EEG förderte Investitionen in erneuerbare Energien](https://www.bundesnetzagentur.de/DE/Fachthemen/ElektrizitaetundGas/ErneuerbareEnergien/ZahlenDatenInformationen/start.html)
- Bundesnetzagentur - [Ansturm auf die Installation von Solarpanels](https://www.bundesnetzagentur.de/DE/Fachthemen/ElektrizitaetundGas/ErneuerbareEnergien/ZahlenDatenInformationen/start.html)
- Strom-Report - [Hoher Akzeptanz und Nachfrage nach Solarenergie in der Bevölkerung](https://strom-report.com/photovoltaik/)
- Erneuerbare Energien - [Redaktion Dossier](https://www.erneuerbare-energien.de/EE/Redaktion/DE/Dossier/eeg.html)
- Wikipedia - [Geschichte der Windenergienutzung](https://de.wikipedia.org/wiki/Geschichte_der_Windenergienutzung)
- ...

"""

# Textblock CO2 Emissionen
text_co2_emissionen_1 = textblock(
    title = "Übersicht über die CO₂ Emissionen in Deutschland, Grossbritannien, der Schweiz und Spanien (1990-2020)",
    text = "Die vorliegenden Diagramme präsentieren einen umfassenden Überblick über die CO₂ Emissionen von vier europäischen Ländern - Deutschland, Grossbritannien, Schweiz und Spanien - über einen Zeitraum von 30 Jahren, von 1990 bis 2020."
)

text_co2_emissionen2 = textblock(
    title = "CO₂ Emissionen pro Kopf",
    text = "Im ersten Diagramm werden die CO₂ Emissionen pro Kopf für jedes Land dargestellt. Hier wird der jährliche CO₂ Ausstoss pro Einwohner in Tonnen angezeigt. Diese Metrik liefert einen direkten Einblick in die durchschnittliche 'Carbon Footprint' eines Einwohners in jedem der betrachteten Länder."
)

text_co2_emissionen3 = textblock(
    title = "CO₂ Emissionen pro Landesfläche",
    text = "Das zweite Diagramm zeigt die CO₂ Emissionen pro Quadratkilometer für jedes Land. Dies gibt uns eine Vorstellung davon, wie dicht die CO₂ Emissionen in den verschiedenen Ländern sind, unabhängig von der Bevölkerungsdichte."
)

text_co2_emissionen4 = textblock(
    title = "Über den gesamten Betrachtungszeitraum hinweg lässt sich ein ermutigender Trend erkennen:",
    text = "Die CO₂ Emissionen haben in allen vier Ländern abgenommen. Dies weist auf erfolgreiche Bemühungen hin, den Kohlenstoffausstoss zu verringern und den Übergang zu einer nachhaltigeren und umweltfreundlicheren Wirtschaftsweise zu gestalten. Diese Diagramme bieten einen umfassenden und vergleichenden Überblick über den CO₂ Ausstoss in diesen Ländern. Sie dienen als wichtiger Indikator für die Fortschritte, die diese Länder bei der Reduzierung ihrer CO₂ Emissionen gemacht haben und bieten wertvolle Einblicke für zukünftige Klimaschutzstrategien."
)

# Textblock Erneuerbare Energien
text_erneuerbare_energien_1 = textblock(
    title = "Vergleich der erneuerbaren Energien",
    text = "In diesem interaktiven Linienplot können Sie die Entwicklungen in der Produktion von erneuerbaren Energien in Deutschland, Grossbritannien, der Schweiz und Spanien nachverfolgen. Es stehen Daten zu den drei wichtigsten Arten von erneuerbaren Energien zur Verfügung: Solarenergie, Wasserkraft und Windenergie."
)

text_erneuerbare_energien_2 = textblock(
    text = "Mit den bereitgestellten Checkboxen können Sie auswählen, welche Art von erneuerbarer Energie in der Grafik angezeigt werden soll. Auf diese Weise können Sie individuell untersuchen, wie sich die Produktion jeder Energieart im Laufe der Zeit in den verschiedenen Ländern entwickelt hat."
)

text_erneuerbare_energien_3 = textblock(
    text = "Der interaktive Plot ermöglicht es Ihnen, die Daten visuell zu erfassen und direkte Vergleiche zwischen den Ländern und den verschiedenen Energiearten zu ziehen. Sie können zum Beispiel die Entwicklung der Solarenergie in Deutschland mit der in Spanien vergleichen, oder betrachten, wie sich die Nutzung der Windenergie in Grossbritannien im Vergleich zur Schweiz entwickelt hat."
)

text_erneuerbare_energien_4 = textblock(
    text = "Diese grafische Darstellung dient als wertvolles Instrument für die Analyse und das Verständnis der Dynamik und Trends im Bereich der erneuerbaren Energien in diesen vier Ländern. Wir hoffen, dass diese Daten Ihnen helfen, fundierte Schlussfolgerungen zu ziehen und ein tieferes Verständnis für den Fortschritt in der Nutzung erneuerbarer Energien zu gewinnen."
)

# Entwicklung der erneuerbaren Energien in Deutschland
text_solar_deutschland = textblock(
    title = "Solarenergie in Deutschland",
    text = ""
)

text_wind_deutschland = textblock(
    title = "Windenergie in Deutschland",
    text = ""
)

text_wasser_deutschland = textblock(
    title = "Wasserkraft in Deutschland",
    text = ""
)

# Entwicklung der erneuerbaren Energien in Grossbritannien
text_solar_grossbritannien = textblock(
    title = "Solarenergie in Grossbritannien",
    text = ""
)

text_wind_grossbritannien = textblock(
    title = "Windenergie in Grossbritannien",
    text = ""
)

text_wasser_grossbritannien = textblock(
    title = "Wasserkraft in Grossbritannien",
    text = ""
)

# Entwicklung der erneuerbaren Energien in der Schweiz
text_solar_schweiz_1 = textblock(
    title = "Warum hat die Schweiz erst später mit der Produktion von Solarenergie begonnen? - oder warum waren sie langsam, verglichen mit anderen Länder?",
    text = """- Früher waren die Kosten für die Installation von Solaranlagen sehr hoch.
- Die begrenzte Verfügbarkeit von Sonnenlicht in einigen Teilen des Landes ist nicht optimal.
- Die Schweiz hat eine stabile Energieversorgung, die hauptsächlich auf Wasserkraft basiert.
- Heute ist die Netzparität erreicht und Solarstrom ist billiger wie früher.
"""
)

text_solar_schweiz_2 = textblock(
    title = "Warum hat die Schweiz im Vergleich zu anderen Länder relativ wenig Solarenergie?",
    text = """- Bergige Regionen erschweren den Einsatz von Solaranlagen
    - Wenig Infrastruktur, Stromleitungen, höhere Kosten
- Landschaftsschutz
- Bürokratie und lange Genehmigungsverfahren verlangsamen den Ausbau
"""
)

text_solar_schweiz_3 = textblock(
    title = "Da wir in der Schweiz teilorts öfters Nebel haben, können Solaranlagen damit umgehen? Lohnt es sich dann überhaupt eine Solaranlage zu installieren?",
    text = """- Generell produzieren Solaranlagen auch im Nebel Strom.
- Investitionen in grosse Solaranlagen an nebeligen Standorten sind nicht empfehlenswert.
- Eine kleine PV-Anlage an sonnigen Standorten kann ökologisch sehr sinnvoll sein
"""
)

text_solar_schweiz_4 = textblock(
    subtitle = "Sonnenschein",
    text = """Anhand dieser Visualisierung kann man erkennen, wo es in der Schweiz am meisten Sonnenschein gibt. Im Allgemeinen gilt: Je südlicher, desto sonniger. Ausserdem zeigt sich, dass es in den Tälern mehr Sonnenschein gibt als auf den Berggipfeln (Rhonental, Engadin, Rheintal).
- Kantone auf der Alpensüdseite
    - Tessin
        - Locarno / Cimetta
    - Graubünden
        - Poschiavo, Brusio
    - Wallis
        - Sion
        - Zermatt, Täsch
"""
)

text_solar_schweiz_5 = textblock(
    subtitle = "Ausbau der Solaranlagen",
    text = """In dieser Grafik sieht man den Fortschritt der jeweiligen Gemeinden bezüglich des Ausbaus von Solaranlagen. Einige Gemeinden liegen vorn, wie Neuendorf, Onnens, Evionnaz und Courgenay, aber auch erkennbare Regionen wie:
- Kanton Luzern
- Kanton Zug
- Kanton Thurgau
- Rheintal
"""
)

text_wind_schweiz = textblock(
    title = "Windenergie in der Schweiz",
    text = ""
)

text_wasser_schweiz = textblock(
    title = "Wasserkraft in er Schweiz",
    text = ""
)

# Entwicklung der erneuerbaren Energien in Spanien
text_solar_spanien = textblock(
    title = "Solarenergie in Spanien",
    text = ""
)

text_wind_spanien = textblock(
    title = "Windenergie in Spanien",
    text = ""
)

text_wasser_spanien = textblock(
    title = "Wasserkraft in Spanien",
    text = ""
)

# Textblock Policy Advice
text_policy_advice_1 = textblock(
    title = "Hypothese 1",
    text = "Die Erfahrung Spaniens dient in anderen Ländern (insbesondere Deutschland und Grossbritannien) als Warnung, dass eine zu grosszügige Einspeisevergütung ohne entsprechende Kontrollmechanismen zu finanziellen Schwierigkeiten führen kann. Eine konkrete Massnahme könnte darin bestehen, regelmässige Überprüfungen und Anpassungen der Einspeisevergütungen durchzuführen, um sicherzustellen, dass sie die Marktbedingungen widerspiegeln und nicht zu übermässigen Kosten führen."
)

text_policy_advice_2 = textblock(
    title = "Hypothese 2",
    text = "Die Implementierung eines Systems wie SEG in Grossbritannien kann in der Schweiz und in Deutschland den Anreiz für Hausbesitzer und kleine Unternehmen erhöhen könnte, in erneuerbare Energiesysteme zu investieren. Dadurch könnte der Anteil der erneuerbaren Energien insgesamt gesteigert werden."
)

# Textblock Policy Advice
text_policy_advice_3 = textblock(
    title = "Hypothese 3",
    text = "Da die Schweiz noch nicht viel Solarenergie erzeugt hat und ein Platzmangel herrscht, wäre eine vielversprechende Möglichkeit, Solaranlagen auf bereits vorhandener Infrastruktur (z.B. Stauseen, Bahngleisen, Strommasten, Bahnstationen, usw.)  zu errichten, ähnlich wie es bereits am Mutsee oder Albignasee geschieht."
)

# Dashboard

In [119]:
width = 900
small_width = 650
margin = 0
template = pn.template.FastListTemplate(
    title = "Policy Advices für eine grüne Zukunft",
    sidebar_width = 375,
    sidebar = [
        pn.pane.Markdown("## Allgemeine Informationen"),
        pn.pane.Markdown(text_allgemeine_infos),
        pn.pane.Markdown("## Einstellungen"),
        pn.pane.Markdown("Mittels Sliders und Buttons ist es möglich die einzelnen Plots auf das Jahr und das Land zu beschränken"),
        pn.Spacer(height=10),
        pn.Row(
            country_selector,
            margin=(0, 5)
        ),
        pn.Row(
            start_year_slider,
            margin=(10, 5)
        ),
        pn.Row(
            end_year_slider,
            margin=(10, 5)
        ),
        pn.pane.PNG('sidebar_image.png', sizing_mode='stretch_both'),
        pn.pane.Markdown("Quelle: [Kasakata](https://kasakata.co.id/wp-content/uploads/2022/03/eco-green-thumbnail-kasakata.jpg)"),
        pn.Spacer(height=30)
    ],
    main = [
        pn.Row(
            pn.Column(
                "# Willkommen!",
                pn.Card(
                    text_einleitung,
                    title = "Einleitung",
                    margin = (5,25),
                    sizing_mode = 'scale_width',
                    collapsed = True
                ),
                pn.Card(
                    text_quellen,
                    title = "Daten und Quellen",
                    margin = (5,25),
                    sizing_mode = 'scale_width',
                    collapsed = True
                ),
                pn.Card(
                    text_sidebar,
                    title = "Anleitung",
                    margin = (5,25),
                    sizing_mode = 'scale_width',
                    collapsed = True
                ),
                pn.Spacer(height=30)
            )
        ),
        pn.Row(
            pn.Column(
                "# Datastory",

                # Co2 Emissionen Plots
                pn.Card(
                    
                    pn.Row(
                        plot_co2.panel(),
                        plot_co2_km2.panel(),
                    ),
                    text_co2_emissionen_1,
                    text_co2_emissionen2,
                    text_co2_emissionen3,
                    text_co2_emissionen4,
                    title = "CO₂ Emissionen",
                    margin = (5,25),
                    sizing_mode = 'scale_width',
                ),

                # Vergleich der Länder (erneuerbare Energien)
                pn.Card(
                    pn.Row(
                        pn.Column(
                            renewable_energy_selector,
                            plot_renewable_energy.panel()
                        ),
                        margin=(10, 0)
                    ),
                    text_erneuerbare_energien_1,
                    text_erneuerbare_energien_2,
                    text_erneuerbare_energien_3,
                    text_erneuerbare_energien_4,
                    title = "Erneuerbare Energien",
                    margin = (5,25),
                    sizing_mode = 'scale_width',
                ),
                # Entwicklung im Zeitverlauf
                pn.Card(

# ---------------- DEUTSCHLAND ----------------

                    pn.Row(
                        pn.Card(
                            text_solar_deutschland,
                            title = "Solarenergie in Deutschland",
                            margin = (5,25),
                            sizing_mode = 'scale_width',
                            collapsed = True
                        ),
                        sizing_mode = 'scale_width',
                    ),
                    pn.Row(
                        pn.Card(
                            text_wind_deutschland,
                            title = "Windenergie in Deutschland",
                            margin = (5,25),
                            sizing_mode = 'scale_width',
                            collapsed = True
                        ),
                        sizing_mode = 'scale_width'
                    ),
                    pn.Row(
                        pn.Card(
                            text_wasser_deutschland,
                            title = "Wasserkraft in Deutschland",
                            margin = (5,25),
                            sizing_mode = 'scale_width',
                            collapsed = True
                        ),               
                    ),

# ---------------- GROSSBRITANNIEN ----------------

                    pn.Row(
                        pn.Card(
                            text_solar_grossbritannien,
                            title = "Solarenergie in Grossbritannien",
                            margin = (5,25),
                            sizing_mode = 'scale_width',
                            collapsed = True
                        ),
                        sizing_mode = 'scale_width'
                    ),
                    pn.Row(
                        pn.Card(
                            text_wind_grossbritannien,
                            title = "Windenergie in Grossbritannien",
                            margin = (5,25),
                            sizing_mode = 'scale_width',
                            collapsed = True
                        ),
                        sizing_mode = 'scale_width'
                    ),
                    pn.Row(
                        pn.Card(
                            text_wasser_grossbritannien,
                            title = "Wasserkraft in Grossbritannien",
                            margin = (5,25),
                            sizing_mode = 'scale_width',
                            collapsed = True
                        ),               
                    ),

# ---------------- SCHWEIZ ----------------

                    pn.Row(
                        pn.Card(
                            text_solar_schweiz_1,
                            text_solar_schweiz_2,
                            text_solar_schweiz_3,
                            pn.Column(
                                pn.pane.Markdown("### Weitere Daten zur Solarenergie in der Schweiz"),
                                pn.Row(
                                    pn.pane.PNG('schweiz_sonnenschein.png', sizing_mode = 'scale_width'),
                                    pn.pane.PNG('schweiz_ausbau_solaranlagen.png', sizing_mode = 'scale_width'),
                                ),
                                pn.Row(
                                    pn.pane.Markdown("Quelle: MeteoSchweiz - Klimanormwerte Sonnenscheindauer 1981-2010", sizing_mode = 'scale_width'),
                                    pn.pane.Markdown("Quelle: Bundesamt für Energie - Ausbau der Solaranlagen 2023", sizing_mode = 'scale_width'),
                                ),
                                pn.Row(
                                    pn.pane.Markdown(text_solar_schweiz_4, sizing_mode = 'scale_width'),
                                    pn.pane.Markdown(text_solar_schweiz_5, sizing_mode = 'scale_width'),
                                ),
                                sizing_mode = 'scale_width'
                            ),
                            title = "Solarenergie in der Schweiz",
                            margin = (5,25),
                            sizing_mode = 'scale_width',
                            #collapsed = True
                        ),
                        sizing_mode = 'scale_width'
                    ),
                    pn.Row(
                        pn.Card(
                            text_wind_schweiz,
                            title = "Windenergie in der Schweiz",
                            margin = (5,25),
                            sizing_mode = 'scale_width',
                            collapsed = True
                        ),
                        sizing_mode = 'scale_width'
                    ),
                    pn.Row(
                        pn.Card(
                            text_wasser_schweiz,
                            title = "Wasserkraft in der Schweiz",
                            margin = (5,25),
                            sizing_mode = 'scale_width',
                            collapsed = True
                        ),               
                    ),

# ---------------- SPANIEN ----------------
                    pn.Row(
                        pn.Card(
                            text_solar_spanien,
                            title = "Solarenergie in Spanien",
                            margin = (5,25),
                            sizing_mode = 'scale_width',
                            collapsed = True
                        ),
                        sizing_mode = 'scale_width'
                    ),
                    pn.Row(
                        pn.Card(
                            text_wind_spanien,
                            title = "Windenergie in Spanien",
                            margin = (5,25),
                            sizing_mode = 'scale_width',
                            collapsed = True
                        ),
                        sizing_mode = 'scale_width'
                    ),
                    pn.Row(
                        pn.Card(
                            text_wasser_spanien,
                            title = "Wasserkraft in Spanien",
                            margin = (5,25),
                            sizing_mode = 'scale_width',
                            collapsed = True
                        ),               
                    ),
                    pn.Spacer(height=30),
                    title = "Entwicklung der erneuerbarer Energie im Zeitverlauf",
                    margin = (5,25),
                    sizing_mode = 'scale_width',
                    collapsed = True
                ),

                # Policy Advices
                pn.Card(
                    pn.Row(
                        

                    ),
                    text_policy_advice_1,
                    text_policy_advice_2,
                    text_policy_advice_3,
                    title = "Policy Advices",
                    margin = (5,25),
                    sizing_mode = 'scale_width',
                    collapsed = True
                ),
                pn.Spacer(height=30),
                sizing_mode = 'scale_width'
            ),
        )
    ],
    accent_base_color = '#2E8B57',
    header_background = '#2E8B57',
)

template.show()

Launching server at http://localhost:54852
